In [3]:
import pyrealsense2 as rs
import numpy as np
import cv2
from statistics import mode
import re
depth_dict={}
def depth_filter(win_data):
    x1,y1,x2,y2=win_data[0]-win_data[2]/2,win_data[1]-win_data[3]/2,win_data[0]+3*win_data[2]/2,win_data[1]+3*win_data[3]/2
    for i in range(int(y1),int(y2)):
        for j in range(int(x1),int(x2)):
            d=aligned_depth_frame.get_distance(int(j),int(i))
            if d!=0:
                k=[j,i]
                depth_dict[f'{k}']=round(d, 2)
    depth_list=depth_dict.values()
    filtered_depth=mode(depth_list)
    print(filtered_depth)
    corr_pix=list(depth_dict.keys())[list(depth_dict.values()).index(filtered_depth)] #get corresponding pixel of the distance value
    print(corr_pix)
    print(type(corr_pix))
    #print(corr_pix) #A string here
    corr_pix=re.split(', ',corr_pix)
    pix_corr=[]
    for num in corr_pix:
        num=re.sub(r"[\([{})\]]", "", num)
        num=int(num)
        pix_corr.append(num)
    
    print(pix_corr)
    print(type(pix_corr))
    a,b=pix_corr[0],pix_corr[1]
    depth=aligned_depth_frame.get_distance(int(a),int(b))
    return depth

def distance(event,x,y,flags,param):
    if event==cv2.EVENT_LBUTTONDOWN:
        print(event)
        cv2.circle(capture,(x,y),2,(128,0,128),-1)
        print((x,y))
        window=[int(x),int(y),10,10]
        d=depth_frame.get_distance(int(x),int(y))
        print(d)
        if (d==0.0) :
            print('Depth is zero, use an window')
            d1=depth_filter(window)
            x_w1, y_w1, z_w1= convert_depth_to_phys_coord_using_realsense(int(x),int(y), d1, camera_info)
            print('Points according to window:',x_w1, y_w1, z_w1)
        else:
            x_w, y_w, z_w = convert_depth_to_phys_coord_using_realsense(int(x),int(y), d, camera_info)
            print('Points are:',x_w, y_w, z_w)
            d1=depth_filter(window)
            x_w1, y_w1, z_w1= convert_depth_to_phys_coord_using_realsense(int(x),int(y), d1, camera_info)
            print('Points according to window:',x_w1, y_w1, z_w1)

def convert_depth_to_phys_coord_using_realsense(x, y, depth, cameraInfo):
    _intrinsics = rs.intrinsics()
    _intrinsics.width = cameraInfo.width
    _intrinsics.height = cameraInfo.height
    _intrinsics.ppx = cameraInfo.ppx
    _intrinsics.ppy = cameraInfo.ppy
    _intrinsics.fx = cameraInfo.fx
    _intrinsics.fy = cameraInfo.fy
    # _intrinsics.model = cameraInfo.distortion_model
    _intrinsics.model  = rs.distortion.none
    _intrinsics.coeffs = [i for i in cameraInfo.coeffs]
    print(_intrinsics)
    result = rs.rs2_deproject_pixel_to_point(_intrinsics, [x, y], depth)
    # result[0]: right, result[1]: down, result[2]: forward
    return result[0], -result[1], -result[2]
"""
# Setup the pipeline
pipe = rs.pipeline()
cfg = rs.config()
cfg.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 30)
cfg.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)
profile = pipe.start(cfg)
"""

# Create a pipeline
pipeline = rs.pipeline()

# Create a config and configure the pipeline to stream
config = rs.config()
pipeline_wrapper = rs.pipeline_wrapper(pipeline)
pipeline_profile = config.resolve(pipeline_wrapper)
device = pipeline_profile.get_device()
device_product_line = str(device.get_info(rs.camera_info.product_line))
print(device_product_line)
found_rgb = False
for s in device.sensors:
    if s.get_info(rs.camera_info.name) == 'RGB Camera':
        found_rgb = True
        print("There is a depth camera with color sensor")
        break
if not found_rgb:
    print("The demo requires Depth camera with Color sensor")
    exit(0)
config.enable_stream(rs.stream.depth, 848,480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 848,480, rs.format.bgr8, 30)
profile = pipeline.start(config)


# Setup the 'High Accuracy'-mode
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()
print("Depth Scale is: " , depth_scale)
clipping_distance_in_meters = 0.25 #1 meter
clipping_distance = clipping_distance_in_meters / depth_scale
print(clipping_distance)
preset_range = depth_sensor.get_option_range(rs.option.visual_preset)
for i in range(int(preset_range.max)):
    visulpreset = depth_sensor.get_option_value_description(rs.option.visual_preset,i)
    print('%02d: %s'%(i,visulpreset))
    if visulpreset == "High Accuracy":
        depth_sensor.set_option(rs.option.visual_preset, i)
# enable higher laser-power for better detection
depth_sensor.set_option(rs.option.laser_power, 180)
# lower the depth unit for better accuracy and shorter distance covered
depth_sensor.set_option(rs.option.depth_units, 0.0005)
align_to = rs.stream.color
align = rs.align(align_to)
# Skip first frames for auto-exposure to adjust
for x in range(5):
    pipeline.wait_for_frames()
try:
    while True:

        # Stores next frameset
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        depth_frame = frames.get_depth_frame()

        if color_frame:
            aligned_frames = align.process(frames)

            # Get aligned frames
            aligned_depth_frame = aligned_frames.get_depth_frame() # aligned_depth_frame is a 640x480 depth image
            color_frame = aligned_frames.get_color_frame()

            # Validate that both frames are valid
            if not aligned_depth_frame or not color_frame:
                continue

            depth_image = np.asanyarray(aligned_depth_frame.get_data())
            color_image = np.asanyarray(color_frame.get_data())

            # Remove background - Set pixels further than clipping_distance to grey
            black_color = 0
            depth_image_3d = np.dstack((depth_image,depth_image,depth_image)) #depth image is 1 channel, color is 3 channels
            bg_removed = np.where((depth_image_3d > clipping_distance) | (depth_image_3d <= 0), black_color, color_image)

            # Render images:
            #   depth align to color on left
            #   depth on right
            #depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
            #images = np.hstack((bg_removed, depth_colormap))
            cv2.namedWindow('depth_cut', cv2.WINDOW_NORMAL)
            cv2.imshow('depth_cut', bg_removed)
            camera_info = aligned_depth_frame.profile.as_video_stream_profile().intrinsics
            if cv2.waitKey(1) & 0xFF==27:
                capture=bg_removed
                cv2.destroyAllWindows()
                break

finally:
    while True:
        cv2.imshow('object',capture)
        cv2.setMouseCallback('object',distance)
        if cv2.waitKey(10) & 0xFF==27:
            cv2.destroyAllWindows()
            break
    # Stop streaming
    pipeline.stop()

D400
There is a depth camera with color sensor
Depth Scale is:  0.0005000000237487257
499.9999762512755
00: Custom
01: Default
02: Hand
03: High Accuracy
04: High Density
1
(130, 255)
0.18950000405311584
[ 848x480  p[429.725 251.306]  f[606.24 605.852]  None [0 0 0 0 0] ]
Points are: -0.09368883818387985 -0.0011555579258129 -0.18950000405311584
0.19
[125, 250]
<class 'str'>
[125, 250]
<class 'list'>
[ 848x480  p[429.725 251.306]  f[606.24 605.852]  None [0 0 0 0 0] ]
Points according to window: -0.09393604099750519 -0.0011586069595068693 -0.1900000125169754
1
(751, 137)
0.2900000214576721
[ 848x480  p[429.725 251.306]  f[606.24 605.852]  None [0 0 0 0 0] ]
Points are: 0.15368449687957764 0.054714057594537735 -0.2900000214576721
0.19
[125, 250]
<class 'str'>
[125, 250]
<class 'list'>
[ 848x480  p[429.725 251.306]  f[606.24 605.852]  None [0 0 0 0 0] ]
Points according to window: 0.1006898432970047 0.03584713861346245 -0.1900000125169754
1
(188, 404)
0.27400001883506775
[ 848x480  p[429.